In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import requests
import re
import time
import concurrent.futures

from tqdm import tqdm
tqdm.pandas()

In [2]:
df = pd.read_csv(r"dataset\transfers_complete.csv")
df["agent"] = None

In [3]:
HEADERS = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'cache-control': 'max-age=0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36 OPR/62.0.3331.116',
}

session = requests.session()
session.headers.update(HEADERS)

def scrape_agent(session, player_id, wait=10):
    time.sleep(wait)
    try:
        url = f"https://www.transfermarkt.com/generic-player/profil/spieler/{player_id}"
        #print(url)
        
        resp = session.get(url)
        #print(resp)

        html = BeautifulSoup(resp.text)
        
        subsection = html.find("div", class_="box viewport-tracking").find("div", class_="large-6 large-pull-6 small-12 columns spielerdatenundfakten")
        agent_section = subsection.find("span", class_="info-table__content info-table__content--bold info-table__content--flex")
        try:
            agent = re.findall(r'><span class="cp" title="'+ "(.*?)" +'">', str(agent_section))[0]
        except IndexError:
            agent = None
        
        # Se il primo metodo fallisce, tenta di estrarre il nome dell'agente dal tag <a> con onclick specifico
        if agent is None:
            agent_link = agent_section.find('a', onclick=re.compile(r'tmEvent\(\"spielerprofil\"'))
            if agent_link:
                agent = agent_link.text.strip()
        
        if agent:
            return agent
        else:
            return "no_agent"

    except Exception as e:
        print(f"Error: {e}")
        return "no_agent"

In [4]:
temp = df[-5:]
temp

,league,season,window,team_id,team_name,team_country,dir,player_id,player_name,player_age,...,counter_team_name,counter_team_country,transfer_fee_amnt,market_val_amnt,is_free,is_loan,is_loan_end,is_retired,transfer_id,agent
82377,NL1,2024,s,724,FC Volendam,Belgium,left,1014243,Vivaldo Semedo,19.0,...,Udinese Calcio,Italy,NaN,500.0,False,True,True,False,4844834,None
82378,NL1,2024,s,724,FC Volendam,Belgium,left,925584,Axel Guessand,19.0,...,Udinese Calcio,Italy,NaN,500.0,False,True,True,False,4832825,None
82379,NL1,2024,s,724,FC Volendam,Belgium,left,746838,Zach Booth,20.0,...,Leicester City U21,England,NaN,500.0,False,True,True,False,4623166,None
82380,NL1,2024,s,724,FC Volendam,Belgium,left,967948,Garang Kuol,19.0,...,Newcastle United,England,NaN,700.0,False,True,True,False,4569682,None
82381,NL1,2024,w,1304,Heracles Almelo,Moldova,in,240889,Anas Ouahim,27.0,...,FC Sheriff Tiraspol,Moldova,NaN,700.0,False,True,True,False,4923979,None


In [5]:
# temp["agent"] = temp["player_id"].progress_apply(lambda x: scrape_agent(session, x, 5))
# temp

In [6]:
df = df[df["is_free"] == False]
df = df[df["is_loan"] == False]
df = df[df["is_loan_end"] == False]
df = df[df["is_retired"] == False]
# df = df[df["season"].isin([2024, 2023,2022,2021,2020])]
df_unique = df.groupby('player_id')

In [7]:
unique_player_ids = df["player_id"].unique()
print(unique_player_ids[:10])
def scrape_agent_wrapper(player_id):
    return player_id, scrape_agent(session, player_id, 2)

[33544 62049 43261 73538  8198 42411 64484 15449  3881  5957]


In [8]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(tqdm(executor.map(scrape_agent_wrapper, unique_player_ids), total=len(unique_player_ids)))
agent_dict = dict(results)
df['agent'] = df['player_id'].map(agent_dict)
df

 28%|██▊       | 3790/13422 [24:34<1:16:39,  2.09it/s]

Error: 'NoneType' object has no attribute 'find'
Error: 'NoneType' object has no attribute 'find'


 28%|██▊       | 3802/13422 [24:37<55:12,  2.90it/s]  

Error: 'NoneType' object has no attribute 'find'


 43%|████▎     | 5722/13422 [38:04<35:05,  3.66it/s]  

Error: 'NoneType' object has no attribute 'find'


 91%|█████████ | 12164/13422 [1:19:55<05:55,  3.54it/s] 

Error: 'NoneType' object has no attribute 'find'


100%|██████████| 13422/13422 [1:28:08<00:00,  2.54it/s]


,league,season,window,team_id,team_name,team_country,dir,player_id,player_name,player_age,...,counter_team_name,counter_team_country,transfer_fee_amnt,market_val_amnt,is_free,is_loan,is_loan_end,is_retired,transfer_id,agent
0,GB1,2009,s,985,Manchester United,England,in,33544,Antonio Valencia,23.0,...,Wigan Athletic,England,18800000.0,NaN,False,False,False,False,310832,JP PACHON SPORTS
1,GB1,2009,s,985,Manchester United,England,in,62049,Mame Diouf,21.0,...,Molde FK,Norway,4500000.0,1600000.0,False,False,False,False,319841,Amin Sakman
2,GB1,2009,s,985,Manchester United,England,in,43261,Gabriel Obertan,20.0,...,FC Girondins Bordeaux,France,4000000.0,400000.0,False,False,False,False,315185,JL Sport Consulting
4,GB1,2009,s,985,Manchester United,England,in,73538,Scott Moffatt,18.0,...,Manchester United U18,England,NaN,NaN,False,False,False,False,339015,no_agent
12,GB1,2009,s,985,Manchester United,England,left,8198,Cristiano Ronaldo,24.0,...,Real Madrid,Spain,94000000.0,45000000.0,False,False,False,False,308498,Gestifute
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82371,NL1,2024,s,724,FC Volendam,Belgium,left,672100,Darius Johnson,24.0,...,Without Club,Without Club,NaN,250.0,False,False,False,False,4958717,Sporting Talent
82372,NL1,2024,s,724,FC Volendam,Belgium,left,508642,Ibrahim El Kadiri,22.0,...,Without Club,Without Club,NaN,300.0,False,False,False,False,4958716,ESA
82373,NL1,2024,s,724,FC Volendam,Belgium,left,638666,Josh Flint,23.0,...,Without Club,Without Club,NaN,450.0,False,False,False,False,4958707,CAA Base Ltd
82374,NL1,2024,s,724,FC Volendam,Belgium,left,474838,Brian Plat,24.0,...,Without Club,Without Club,NaN,600.0,False,False,False,False,4958706,no_agent


In [9]:
df.to_csv("dataset/transfers_with_agents.csv", index=False)

In [10]:
df["agent"].unique()

array(['JP PACHON SPORTS', 'Amin Sakman', 'JL Sport Consulting', ...,
       'Meforyo Sportsmanagement', 'Dries Management',
       'Forwardzone Football Limited'], dtype=object)

In [17]:
df["agent"].value_counts()

agent
no_agent                        8618
Wasserman                        557
CAA Stellar                      424
Unique Sports Group              329
Gestifute                        271
                                ... 
Bpoint                             1
Rabona Sports                      1
Youonegoal                         1
Mostar Sports Management           1
Forwardzone Football Limited       1
Name: count, Length: 2097, dtype: int64